In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Calculate the radial basis functions for each input sample and each RBF center
        diff = K.expand_dims(inputs) - self.centers  # shape: (batch_size, units, input_dim)
        norm = K.sum(K.square(diff), axis=-1)  # shape: (batch_size, units)
        rbf = K.exp(-self.gamma * norm)  # shape: (batch_size, units)

        return rbf

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units
    
url = 'https://raw.githubusercontent.com/ravsssh/HandballPrediction/main/Handball%20Prediction/datashotiqr/CB.csv'
df = pd.read_csv(url, delimiter=',')
# Menentukan input(X) dan output (y)
X = df.drop(columns=["SHOT","GOAL","7MACC","6MACC","9MACC","WINGACC","PIVOTACC","FASTBREAKACC"])
y = df.GOAL
# Membagi dataset 80:20 untuk train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# normalisasi dataset agar tidak bias
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

validation_results_df = pd.DataFrame(columns=['R2_Test', 'MSE', 'MAE'])

for _ in range(20):
    model = Sequential()
    model.add(Flatten(input_shape=(6,)))
    model.add(RBFLayer(6, 0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='rmsprop', loss=mean_squared_error, metrics=['mean_absolute_error']) 
    history = model.fit(X_train, y_train, batch_size=8, epochs=100, validation_data=(X_test, y_test))
    
    pred_test = model.predict(X_test)
    
    r2_test = r2_score(y_test, pred_test)
    mse = history.history['val_loss'][-1]  # Last MSE from validation data
    mae = history.history['val_mean_absolute_error'][-1]  # Last MAE from validation data
    
    print(f"R-Squared Test Score: {r2_test}")
    
    # Append results to the validation results DataFrame
    validation_results_df = validation_results_df.append({'R2_Test': r2_test,
                                                          'MSE': mse,
                                                          'MAE': mae}, ignore_index=True)
    
    # Reset the TensorFlow session to clear the model
    tf.keras.backend.clear_session()

# Save the validation results DataFrame to a CSV file
validation_results_df.to_csv('validation_resultsshot.csv', index=False)


Epoch 1/100
264/264 [==============================] - 1s 2ms/step - loss: 7.5052 - mean_absolute_error: 1.9816 - val_loss: 5.1550 - val_mean_absolute_error: 1.6660
Epoch 2/100
264/264 [==============================] - 0s 2ms/step - loss: 4.3604 - mean_absolute_error: 1.6016 - val_loss: 4.1888 - val_mean_absolute_error: 1.5737
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 3.7825 - mean_absolute_error: 1.5261 - val_loss: 3.8421 - val_mean_absolute_error: 1.5139
Epoch 4/100
264/264 [==============================] - 0s 2ms/step - loss: 3.5086 - mean_absolute_error: 1.4714 - val_loss: 3.6129 - val_mean_absolute_error: 1.4734
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 3.3001 - mean_absolute_error: 1.4279 - val_loss: 3.4145 - val_mean_absolute_error: 1.4370
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 3.1272 - mean_absolute_error: 1.3891 - val_loss: 3.2532 - val_mean_absolute_error: 1.3955
Epoch 7/10

C:\Users\Wandi\AppData\Local\Temp\ipykernel_120116\1078879232.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


264/264 [==============================] - 1s 2ms/step - loss: 4.7165 - mean_absolute_error: 1.7079 - val_loss: 4.8515 - val_mean_absolute_error: 1.7314
Epoch 2/100
264/264 [==============================] - 0s 1ms/step - loss: 4.4148 - mean_absolute_error: 1.6736 - val_loss: 4.6157 - val_mean_absolute_error: 1.7152
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 4.1952 - mean_absolute_error: 1.6497 - val_loss: 4.3664 - val_mean_absolute_error: 1.6524
Epoch 4/100
264/264 [==============================] - 0s 1ms/step - loss: 3.9775 - mean_absolute_error: 1.6016 - val_loss: 4.1514 - val_mean_absolute_error: 1.6095
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 3.7826 - mean_absolute_error: 1.5575 - val_loss: 3.9500 - val_mean_absolute_error: 1.5948
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 3.6035 - mean_absolute_error: 1.5268 - val_loss: 3.7593 - val_mean_absolute_error: 1.5563
Epoch 7/100
264/264 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_120116\1078879232.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


264/264 [==============================] - 1s 2ms/step - loss: 7.0097 - mean_absolute_error: 1.9031 - val_loss: 5.4722 - val_mean_absolute_error: 1.7439
Epoch 2/100
264/264 [==============================] - 0s 1ms/step - loss: 4.4894 - mean_absolute_error: 1.6337 - val_loss: 4.3939 - val_mean_absolute_error: 1.6262
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 3.9039 - mean_absolute_error: 1.5629 - val_loss: 3.9870 - val_mean_absolute_error: 1.5722
Epoch 4/100
264/264 [==============================] - 0s 1ms/step - loss: 3.6272 - mean_absolute_error: 1.5207 - val_loss: 3.7567 - val_mean_absolute_error: 1.5347
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 3.4568 - mean_absolute_error: 1.4888 - val_loss: 3.5851 - val_mean_absolute_error: 1.4988
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 3.3065 - mean_absolute_error: 1.4542 - val_loss: 3.4357 - val_mean_absolute_error: 1.4849
Epoch 7/100
264/264 [=

...

C:\Users\Wandi\AppData\Local\Temp\ipykernel_120116\1078879232.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


264/264 [==============================] - 1s 2ms/step - loss: 5.4194 - mean_absolute_error: 1.7717 - val_loss: 4.8946 - val_mean_absolute_error: 1.7011
Epoch 2/100
264/264 [==============================] - 0s 1ms/step - loss: 4.1492 - mean_absolute_error: 1.5989 - val_loss: 4.0544 - val_mean_absolute_error: 1.5655
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 3.5633 - mean_absolute_error: 1.4880 - val_loss: 3.6219 - val_mean_absolute_error: 1.4978
Epoch 4/100
264/264 [==============================] - 0s 1ms/step - loss: 3.2878 - mean_absolute_error: 1.4384 - val_loss: 3.3923 - val_mean_absolute_error: 1.4464
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 3.1051 - mean_absolute_error: 1.3981 - val_loss: 3.2220 - val_mean_absolute_error: 1.4229
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 2.9567 - mean_absolute_error: 1.3640 - val_loss: 3.0721 - val_mean_absolute_error: 1.3926
Epoch 7/100
264/264 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_120116\1078879232.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


264/264 [==============================] - 1s 2ms/step - loss: 5.6437 - mean_absolute_error: 1.7813 - val_loss: 5.0411 - val_mean_absolute_error: 1.7257
Epoch 2/100
264/264 [==============================] - 0s 1ms/step - loss: 4.3661 - mean_absolute_error: 1.6437 - val_loss: 4.3588 - val_mean_absolute_error: 1.6270
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 3.8825 - mean_absolute_error: 1.5610 - val_loss: 3.9733 - val_mean_absolute_error: 1.5523
Epoch 4/100
264/264 [==============================] - 0s 1ms/step - loss: 3.5990 - mean_absolute_error: 1.4999 - val_loss: 3.7285 - val_mean_absolute_error: 1.5262
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 3.4094 - mean_absolute_error: 1.4717 - val_loss: 3.5457 - val_mean_absolute_error: 1.4829
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 3.2476 - mean_absolute_error: 1.4385 - val_loss: 3.3882 - val_mean_absolute_error: 1.4488
Epoch 7/100
264/264 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_120116\1078879232.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,
